In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}


In [10]:
task = '''
       Write a concise but engaging blogpost about
       Machine Learning. Make sure the blogpost is
       within 100 words.
       '''


In [11]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise blogpost (with title) on given topics. You must polish your writing based on the feedback you receive and give a refined version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

[autogen.oai.client: 05-29 20:22:16] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [12]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [13]:
print(reply)

Title: "Demystifying Machine Learning: A Quick Guide"

Are you curious about how Netflix suggests your next binge-watch or how your phone unlocks with your face? It's all thanks to Machine Learning! This cutting-edge technology enables computers to learn from data and make intelligent decisions without being explicitly programmed. From self-driving cars to virtual assistants, the applications are endless. Dive into the world of Machine Learning and discover how it powers the innovative tech shaping our future. Join the revolution of smart technology and unlock endless possibilities with Machine Learning. Let's demystify this fascinating field together!


Adding Reflection

In [14]:
# crtic agent to reflect on the work of the writer agent

critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of the writer and provide constructive feedback alongwith a bit of negatives to help improve the quality of the content.",
)

[autogen.oai.client: 05-29 20:22:20] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [15]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

Critic (to Writer):


       Write a concise but engaging blogpost about
       Machine Learning. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Demystifying Machine Learning: A Quick Guide"

Are you curious about how Netflix suggests your next binge-watch or how your phone unlocks with your face? It's all thanks to Machine Learning! This cutting-edge technology enables computers to learn from data and make intelligent decisions without being explicitly programmed. From self-driving cars to virtual assistants, the applications are endless. Dive into the world of Machine Learning and discover how it powers the innovative tech shaping our future. Join the revolution of smart technology and unlock endless possibilities with Machine Learning. Let's demystify this fascinating field together!

------------------------------------------------------------------------------

Nested Chat

In [16]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for your ability to optimize content for search engines,ensuring that it ranks well and attracts organic traffic. Make sure your suggestion is concise (within 3 bullet points), concrete and to the point. Begin the review by stating your role.",
)


[autogen.oai.client: 05-29 20:26:05] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [18]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for your ability to ensure that content is legally compliant and free from any potential legal issues. Make sure your suggestion is concise (within 3 bullet points), concrete and to the point. Begin the review by stating your role.",
)

[autogen.oai.client: 05-29 20:26:47] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [19]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for your ability to ensure that content is ethically sound and free from any potential ethical issues. Make sure your suggestion is concise (within 3 bullet points), concrete and to the point. Begin the review by stating your role. ",
)

[autogen.oai.client: 05-29 20:27:39] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [20]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review the work of other reviewers and give a final suggestion on the content.",
)

[autogen.oai.client: 05-29 20:28:04] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [24]:
def reflection_message(recipient, messages, sender, config):
    msg = recipient.chat_messages_for_summary(sender)
    last_msg_content=msg[-1]['content']
    return f'''Review the following content. 
            \n\n {last_msg_content}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [25]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [26]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


       Write a concise but engaging blogpost about
       Machine Learning. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Demystifying Machine Learning: A Quick Guide"

Are you curious about how Netflix suggests your next binge-watch or how your phone unlocks with your face? It's all thanks to Machine Learning! This cutting-edge technology enables computers to learn from data and make intelligent decisions without being explicitly programmed. From self-driving cars to virtual assistants, the applications are endless. Dive into the world of Machine Learning and discover how it powers the innovative tech shaping our future. Join the revolution of smart technology and unlock endless possibilities with Machine Learning. Let's demystify this fascinating field together!

------------------------------------------------------------------------------

In [27]:
print(res.summary)

Title: "Unveiling Machine Learning: Your Essential Guide"

Unlock the secrets behind Machine Learning, the driving force behind self-driving cars, virtual assistants, and personalized recommendations. Discover how this revolutionary technology learns from data to make autonomous decisions. Join the tech evolution and explore the endless possibilities of Machine Learning. From predictive analytics to image recognition, its applications are boundless. Let's delve into the realm of Machine Learning together and demystify this complex field. Embrace the power of intelligent machines shaping our future!
